In [2]:
import os

In [3]:
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file 

'c:\\ProjectML\\ProjectML_with_MLFlow\\research'

In [4]:
os.chdir("../")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../") 

In [5]:
%pwd

'c:\\ProjectML\\ProjectML_with_MLFlow'

In [6]:
import pandas  as pd


In [7]:
data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv") # this path i got it from the config.yaml file  from the data_validation unzip_data_dir varaible 

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


### EDA 

In [9]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [10]:
data.shape

(1599, 12)

In [11]:

data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [12]:
# lets prepare the entity here , and if i open the config.yaml file where i can find the root_dir,status_file,unzip_data the same i do find in the class iside the DataValiadationConfig
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict # here all_Schema just read all the data and install inside the all_schema varaible as a dictionary formate

In [13]:
# in order to prepare my cofiguration manager in src config i need this below 2 packages 
from PROJECTML.constants import *
from PROJECTML.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager: # this is components name 
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath) # here iam reading these all like config,params,schema 
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

# then i will prepare my get_data_validation_config , so inside this iam returning all of the varaibles that i have deifined inside my entity 
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation  # after reading by config iam returning the root_dir,status_file etc one by one
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig( # the above entity code is return type , and the below varaibles are getting return after reading by config varaible 
            root_dir=config.root_dir, 
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [15]:
# Now iam going to create the components 
# First i will import some things 
import os
from PROJECTML import logger

In [16]:
# Now i will create DataValidation class 

class DataValiadtion: # this is components name 
    def __init__(self, config: DataValidationConfig): # it will take my DataValidationConfig
        self.config = config

# now iam below iam going to create validate_all_columns,this is a simple python program
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir) # here iam reading the dataset
            all_cols = list(data.columns) # here iam list down all the columns,it will check or matches the schema.yaml file columns whether all the columns present in schema.yaml file columns  present or not 

            all_schema = self.config.all_schema.keys()

            # if those schema.yaml file columns are present it will return status as true, else it will returnt he status as false, then it will return one txt file inside the artifacts folder which theat txt file contains status decision true or false
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [20]:
try:
    config = ConfigurationManager() # here iam initializing my configuration manager
    data_validation_config = config.get_data_validation_config() # here iam getting my get_data_validation_config
    data_validation = DataValiadtion(config=data_validation_config) # then iam calling my DataValidation class and passing data_validation_config inside it
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-10 21:39:28,865: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-10 21:39:28,868: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 21:39:28,879: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-10 21:39:28,879: INFO: common: created directory at: artifacts]
[2023-12-10 21:39:28,879: INFO: common: created directory at: artifacts/data_validation]
